# Types in Julia

In this notebook we will start exploring types by implementing a type to implement algorithmic differentiation.

### What is a type?

A *type* can be thought of as a label that is associated with data stored in memory; this label tells Julia how to interpret the data. For example:

In [1]:
x = 1
y = 1.3

1.3

In [2]:
sizeof(x), sizeof(y)

(8, 8)

In [3]:
typeof(x), typeof(y)

(Int64, Float64)

Both variables are stored in 8 bytes (64 bits), but one is interpreted as an integer and the other as a floating-point number.  Similarly, a pair of two numbers may be intepreted as a complex number, or an interval, or a dual number, or...; although the same information may be stored (two numbers), we want each of these different *kinds* or *types* of objects to be treated differently. 

## Algorithmic differentiation

In the previous notebook we used `ForwardDiff.jl` to automatically differentiate a function. Here we will see how to implement a simple version of this.

The idea is to approximate a (nice enough) function $f$ near a point $a$ by a Taylor series of order 1, i.e. a straight line passing through $(a, f(a))$, with slope equal to the derivative $f'(a)$:
    
$f(x) \simeq f(a) + \epsilon f'(a)$,

where $\epsilon := x - a$.

We now use this to derive the standard rules for the derivative of a sum and product:

$$f(x) + g(x) \simeq [f(a) + g(a)] + \epsilon [f'(a) + g'(a)]$$

$$f(x) \cdot g(x) \simeq [f(a) \cdot g(a)] + \epsilon [f(a) g'(a) + g(a) f'(a)],$$

where we suppose that $\epsilon$ is small enough that $\epsilon^2 = 0$, or alternatively just "take the linear part".

### Defining a composite type

We see that by using just two pieces of information, namely the value $f(a)$ and the derivative $f'(a)$, we can represent a function $f$ near a given point $a$. 

The pair $(f(a), f'(a))$ is often called a **dual number**. We see that is has certain **behaviours** under arithmetic operations. Whenever we have a new behaviour, a *new type is lurking*!

We group the two values into a **composite type**. We can think of a composite type as specifying the structure of a box containing several pieces of information (data) inside. Defining a composite type with two **fields** (pieces of information) has the following syntax:

In [5]:
struct MyType
    a
    b::Int
end

Here we have additionally specified that the information stored in the field `b` must be of type `Int` using the **type annotation operator**, `::`.

Creating an object of that type is accomplished as follows:

In [7]:
x = MyType(3, 4)

MyType(3, 4)

We can extract information as follows:

In [8]:
x.a

3

#### Exercise 1

1. Define a composite type `Dual` with fields `value` and `deriv` of type `Float64`.


2. Create two `Dual` numbers `x` and `y`.


3. What happens if you try to add `x` and `y` together?


4. Make a function `add` that adds `x` and `y` and returns a new `Dual` number, following the rules we found above.

## Implementing arithmetic for a type

We would like to be able to use `+` and `*` for our new `Dual` type, rather than typing `add(x, y)`. To do so, we need to do the following

In [10]:
import Base: +, *

In Julia, `+` and `*` are just functions. They are defined in `Base` (a module containing basic function definitions) and must be `import`ed before being **extended**. They consist of many different **methods** (versions):

In [11]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:76
[43]

We can add more methods that work on our own types. (We are not allowed to modify their behaviour on combinations of types that to not contain our user-defined types; doing so is known as "type piracy" and can affect other people's code in unexpected ways.)

#### Exercise 2

1. Import the `+` and `*` functions from `Base` and implement them for the `Dual` type.
They should return a new `Dual` object.


2. Check that the number of methods has changed. 


3. Use `@which x + y` to check that Julia knows which method to use when adding two `Dual`s.


4. Can you define `x + a` for a `Dual` number `x` and a real number `a`? What happens 

Amazingly, we now have enough to be able to differentiate simple Julia functions involving only `+` and `*`. Define

In [13]:
a = 3.0
x = Dual(a, 1.0)

UndefVarError: UndefVarError: Dual not defined

We initialize the derivative as 1.0 when we make a `Dual`. If we use `x` then we automatically differentiate!

#### Exercise 3

1. Define `a = 3.0` and `xx = Dual(a, 1.0)`.

    (i) Compute `xx + xx`. The result should have the value $2a$ and the derivative $2$ -- write a test that it does so.
    
    (ii) Do the same for `xx * xx`. 
    
    
2. Define the function `f(x) = x * x + x`. Compute `f(xx)` and check that it gives the correct value and derivative!


3. Does this work for the function `f(x) = x^2 + x`?  What do you need to do?


4. What happens for `f(x) = x^2 + 2x`? What do you need to do?


5. What should you do for `f(x) = sin(x) + x`?

#### Exercise 4

1. Define a function `differentiate` that differentiates a function `f` at a point `a` using `Dual` numbers, by following the above pattern. (It should return just the derivative at the given point.)

This is the basis of ("forward-mode") automatic differentiation. The `ForwardDiff.jl` method contains a sophisticated implementation of this method.

### Parametric types

For simplicity, in the above we fixed the fields in the `Dual` type to be of type `Float64`. By doing so we are actually *losing power*. Instead we should let Julia "fill in" the types. 

To do so, we specify that we want to use a **type parameter** `T`. We can think of this as a "special kind of variable" that can only take on certain kinds of values. We specify this with the following syntax: 

In [14]:
struct MyType2{T}
    a::T
    b::T
end

[Note that we have not reused the name `MyType` since Julia *does not allow types to be redefined in a different way*.]

Here we are specifying that both fields `a` and `b` must share the same type `T`, but we have not restricted what values `T` can take. When we create an object, Julia will *infer* (work out) the type:

In [15]:
x = MyType2(3, 4)

MyType2{Int64}(3, 4)

In [16]:
y = MyType2(3.1, 4.2)

MyType2{Float64}(3.1, 4.2)

Note that `x` and `y` have *different* types.

We can define functions acting on parametric types without necessarily talking about the type parameter:

#### Exercise 5

1. Define a function that takes an object of type `MyType2`, *without* mentioning the type parameter, and returns the sum of the two fields.

   What happens when you apply this function to `x` and `y`?
   
   
2. Define a type `Dual2` with a type parameter `T` and the same functions `+` and `*` as before.


3. Define the function `f(x) = x * x + x`. What happens if you pass in `Dual` numbers with different type parameters?